## Basic Setting

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
sc.stop()

In [5]:
import numpy as np
import re
import argparse
import pyspark
from pyspark.sql import Row
from datetime import datetime

from pyspark.sql import SparkSession
import matplotlib.pyplot as mplt
import matplotlib.ticker as mtick

from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, unix_timestamp, to_date
from pyspark.sql.functions import year,month,dayofmonth
from pyspark.sql import Window
import pyspark.sql.functions as psf
from pyspark.sql import functions as F
from pyspark.sql.functions import col
import pandas as pd
from pyspark.sql import SparkSession

from pyspark.sql.functions import when

sc = SparkContext()
sqlContext = SQLContext(sc)

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import pycountry_convert
import pycountry
import re
import argparse
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark.sql import types
import matplotlib.pyplot as mplt
import matplotlib.ticker as mtick
from pyspark.sql.functions import col, unix_timestamp, to_date

## Load row data

In [ ]:
volume = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
inferschema='true').load('/public/ygong15/xerox/volume.csv')

asset=sqlContext.read.format('com.databricks.spark.csv').options(header='true', sep = '\t', nullValue='NULL',ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True,
inferschema='true').load('/public/ygong15/xerox/AssetData.csv')

## Volume data part 
##### extract useful info
##### Volume Analysis Data for Tableau analysis in python_code_code Part 2.ipynb

In [ ]:
volume=volume.withColumn('dim_asset_inscope', volume['dim_asset_inscope'].cast(IntegerType()))
volume=volume.withColumn('dim_asset_isdeleted', volume['dim_asset_isdeleted'].cast(IntegerType()))
volume=volume.withColumn('dim_acct_sk', volume['dim_acct_sk'].cast(IntegerType()))
volume=volume.fillna({'dim_asset_inscope':0})
volume=volume.fillna({'dim_asset_isdeleted':0})
volume=volume.where(col("dim_acct_sk").isNotNull())

### Find the client who are only inscope or outscope

In [ ]:
volume_1=volume[volume['dim_asset_inscope']==1]
volume_0=volume[volume['dim_asset_inscope']==0]

in_and_out_scope=volume.groupBy('dim_acct_sk','dim_asset_inscope').count().sort('dim_acct_sk')
in_and_out_scope=in_and_out_scope.where(col("dim_acct_sk").isNotNull())
in_or_out_count=in_and_out_scope.groupBy('dim_acct_sk').count()
only_in_or_out=in_or_out_count[in_or_out_count['count']==1]
a=only_in_or_out.join(in_and_out_scope,['dim_acct_sk'])
tmp=a.select('dim_acct_sk','dim_asset_inscope')
client_outscope=tmp[tmp['dim_asset_inscope']==0]
client_inscope=tmp[tmp['dim_asset_inscope']==1]

In [ ]:
client_inscope_list=client_inscope.select('dim_acct_sk').rdd.flatMap(lambda x: x).collect()
client_outscope_list=client_outscope.select('dim_acct_sk').rdd.flatMap(lambda x: x).collect()


### Select clients which exist volume are out of scope

In [ ]:
volume_has_out_scope=volume.filter(~volume.dim_acct_sk.isin(client_inscope_list))

In [ ]:
tmp=volume_has_out_scope.select('dim_acct_sk').rdd.flatMap(lambda x: x).collect()
client_has_outscope_list=list(set(tmp))

#### save as volume_has_out_scope.csv

In [ ]:

volume_has_out_scope.coalesce(1).write.option("inferSchema","true").csv("/public/xye6/Xerox/volume_has_out_scope.csv",header = 'true')




## Asset data part 
### (impute missing values)

###### maxppm1108.csv 
###### Asset Analysis Data (for tableau analysis)

### load data

In [ ]:
# check null value 
def is_missing (df):
    # df type : spark data frame
    df_columns = [i for i in df.columns]
    total_missing_cnt = 0  
    missing_value = []
    for i in df_columns:
        missing_value.append(df.filter(df[i].isNull()).count())
        #print ('column ' + str(i) + ' has ' + str(missing_cnt) + ' missing values')
    total_missing_cnt = reduce(lambda x,y: x+y, missing_value)
    print  ('total number of missing cells over entire dataframe is ' + str(total_missing_cnt))
    return dict(zip(df_columns,missing_value))
# lower string letters
lowercase_func = F.udf(lambda x : x.lower() if x is not None else x,types.StringType())

In [ ]:
# create pyspark instance 
spark = SparkSession.builder.appName('My App').config("spark.some.config.option","some-value").getOrCreate()
# read as spark dataframe 
asset = spark.read.csv(path='/public/ygong15/xerox/AssetData.csv', header = True, sep='\t', inferSchema=True, nullValue='NULL',
                       ignoreLeadingWhiteSpace=True,ignoreTrailingWhiteSpace=True)
asset = asset.drop('dim_asset_id','dim_model_manf_is_xerox_vs_nonxerox','dim_model_is_color_vs_mono','dim_last_status_date')
# change dim_modified_dt_sk to timestamp
asset = asset.withColumn('dim_modified_dt_sk', 
                   to_date(unix_timestamp(col('dim_modified_dt_sk'), 'yyyyMMdd').cast("timestamp")))
# drop the rows that the entire cells are null 
asset2 = asset.dropna(how='all')
asset2 = asset2.filter(asset2.dim_model_nm.isNotNull())
asset2 = asset2.withColumn('dim_model_nm', lowercase_func(asset2.dim_model_nm))


## Split into two dataframes
asset_month = asset2.filter(asset2.dim_asset_isdeleted == 0).\
select('dim_modified_dt_sk','dim_asset_sk','dim_acct_sk','dim_asset_inscope')
## nature dataframe 
asset_nature = asset2.select('dim_asset_sk','dim_acct_sk','dim_model_nm','dim_model_manf_nm','dim_model_is_color',
                             'dim_model_ppm','dim_model_manf_is_xerox','dim_loc_country','dim_loc_state',
                             'dim_loc_city','dim_loc_region_name','dim_loc_iso_country_name')
asset_nature = asset_nature.dropDuplicates()

### impute ppm part

In [ ]:
asset_nature = asset_nature.select('*',F.when(asset_nature.dim_model_ppm.isNull(),0).otherwise(asset2.dim_model_ppm).alias('ppm'))
asset_nature = asset_nature.drop('dim_model_ppm')
model_nm_df = asset_nature.filter(asset_nature.ppm != 0).select('dim_model_nm','ppm')
model_nm_df = model_nm_df.dropDuplicates()
model_nm_df2 = model_nm_df.groupBy('dim_model_nm').count()
##multi-ppm 
multi_ppm = model_nm_df2.filter(model_nm_df2['count'] > 1).orderBy('count', ascending = False)
multi_ppm_model = multi_ppm.select('dim_model_nm').rdd.map(lambda x : x[0]).collect()
len(multi_ppm_model)

In [ ]:
## extract model with mono ppm 
mono_ppm_model = model_nm_df2.filter(model_nm_df2['count'] == 1).select('dim_model_nm').rdd.map(lambda x : x[0]).collect()
mono_ppmDF = model_nm_df.filter(model_nm_df.dim_model_nm.isin(mono_ppm_model))
print('num of rows = {i}'.format(i = mono_ppmDF.count()))
## mono_ppm into a dictionary 
k_mono = mono_ppmDF.select('dim_model_nm').rdd.map(lambda x : x[0]).collect()
v_mono = mono_ppmDF.select('ppm').rdd.map(lambda x : x[0]).collect()
mono_ppm_dict = dict(zip(k_mono,v_mono)) 
print len(mono_ppm_dict) 

In [ ]:
## extract multi ppm by manufactuer 
df_manf = asset_nature.filter(asset_nature.ppm != 0).select('dim_model_nm','ppm','dim_model_manf_nm')
df_manf = df_manf.dropDuplicates()
multi_modelDF = df_manf[df_manf.dim_model_nm.isin(multi_ppm_model)] 
model_manf_join = multi_modelDF.withColumn('model_manf',F.concat_ws('<>',multi_modelDF.dim_model_nm,multi_modelDF.dim_model_manf_nm))

## multi_ppm into dict with --- max ppm
model_manf_maxppm = model_manf_join.groupBy('model_manf').agg(F.max('ppm').alias('max_ppm'))
k_mm =model_manf_maxppm.select('model_manf').rdd.map(lambda x : x[0]).collect()
v_mm = model_manf_maxppm.select('max_ppm').rdd.map(lambda x : x[0]).collect()
model_manf_dict = dict(zip(k_mm,v_mm))
len(model_manf_dict)

In [ ]:
ppm_mono_impute = F.udf(lambda x : mono_ppm_dict[x] if x in mono_ppm_dict.keys() else 0, types.IntegerType())
ppm_multi_impute = F.udf(lambda x : model_manf_dict[x] if x in model_manf_dict.keys() else 0, types.IntegerType())

In [ ]:
assetppm = asset_nature.withColumn('model_manf',F.concat_ws('<>',asset_nature.dim_model_nm,asset_nature.dim_model_manf_nm))
ppm_exist_df = assetppm.filter(assetppm.ppm != 0)
ppm_zero_df = assetppm.filter(assetppm.ppm == 0)

In [ ]:
ppm_zero_df = ppm_zero_df.withColumn('ppm2', ppm_mono_impute(ppm_zero_df.dim_model_nm))
ppm_zero_df = ppm_zero_df.withColumn('ppm3', F.when(ppm_zero_df.ppm2 ==0, ppm_multi_impute(ppm_zero_df.model_manf)).otherwise(0))
ppm_zero_df = ppm_zero_df.withColumnRenamed('ppm3', 'dim_model_ppm')
ppm_zero_df = ppm_zero_df.drop('ppm2','ppm','model_manf')
ppm_exist_df = ppm_exist_df.drop('model_manf')
ppm_exist_df = ppm_exist_df.withColumnRenamed('ppm', 'dim_model_ppm')
df_concat = ppm_exist_df.union(ppm_zero_df)
# df_concat.count() # 9165374

### Impute location part

In [ ]:
country_iso3_dict = {u'BRB':'BRB',
 u'NDL':'NLD',
 u'ABW':'ABW',
 u'URY':'URY',
 u'BOT':'BWA',
 u'INACTIVE':None,
 u'LCA':'LCA',
 u'HRV':'HRV',
 u'ROM':'ROU',
 u'CG': 'COG',
 u'LI':'LIE',
 u'SIN': 'SGP',
 u'Outside North America': None,
 u'SLO': 'SVN',
 u'LES':'LSO',
 u'GB': 'GBR',
 u'SYC':'SYC',
 u'MRI':'MUS',
 u'IRQ':'IRQ',
 u'United Arab Emirates - Do Not Use':'ARE',
 u'CUW':'CUW',
 u'DEN':'DNK',
 u'NIC':'NIC',
 u'US':'USA',
 u'GGY':'GGY',
 u'LBA':'LBY',
 u'BIH':'BIH',
 u'france':'FRA',
 u'AL':'ALB',
 u'MWI':'MWI',
 u'UNKNOWN':None,
 u'ZZZDoNotUse_UK':'GBR',
 u'CH':'CHE',
 u'SWZ1':'SWZ',
 u'CIV':'CIV',
 u'MON':'MNG',
 u'ANGOLA':'AGO',
 u'GER':'DEU',
 u'UK':'GBR',
 u'UAE':'ARE',
 u'LS':'LSO',
 u'GD':'GRD',
 u'XXK':'XKX',
 u'CRO':'HRV',
 u'\x8aLTU':'LTU',
 u'BES':'BES',
 u'GAB':'GAB',
 u'BE':'BEL',
 u'Dubai':'ARE',
 u'Praag':'CZE',
 u'PORTUGAL':'PRT',
 u'MKD':'MKD',
 u'ZAM':'ZMB',
 u'BSD':'BHS',
 u'TZA':'TZA',
 u'BB':'BRB',
 u'JEY':'JEY',
 u'Mauritius':'MUS',
 u'United Kingdom':'GBR',
 u'Netherlands':'NLD'}


def convert_cty(iso3):
    c = pycountry_convert.country_alpha2_to_continent_code(pycountry_convert.country_alpha3_to_country_alpha2(iso3))
    return pycountry_convert.convert_continent_code_to_continent_name(c)

iso_imupte = F.udf(lambda x : country_iso3_dict[x] if x in country_iso3_dict.keys() else x,types.StringType())
region_impute = F.udf(lambda x : convert_cty(x) if x != None and x != 'XKX' else x, types.StringType())

asset_iso = df_concat.select('*',
                         F.when(df_concat.dim_loc_iso_country_name.isNull(), iso_imupte(df_concat.dim_loc_country))\
                          .otherwise(df_concat.dim_loc_iso_country_name).alias('country_iso3'))

asset4 = asset_iso.drop('dim_loc_iso_country_name','dim_loc_country')
asset4 = asset4.withColumn('region', F.when(asset4.country_iso3 =='XKX', 'Europe').otherwise(region_impute(asset4.country_iso3)))

usa_state = asset4.filter(asset4.country_iso3 == 'USA').select('dim_loc_state').distinct().rdd.map(lambda x : x[0]).collect()
impute_us_state_dict = {u'Hawaii':'HI', u'Texas':'TX',u'Pennsylvania':'PA',u'Illinois':'IL',u'Georgia':'GA',u'California':'CA'}
impute_state_us= F.udf(lambda x :impute_us_state_dict[x] if x in impute_us_state_dict.keys() else x, types.StringType())
asset4 = asset4.select('*',F.when(asset4.country_iso3 == 'USA', impute_state_us(asset4.dim_loc_state))
                                  .otherwise(asset4.dim_loc_state).alias('state')) 
asset5 = asset4.drop('dim_loc_state','dim_loc_region_name')
# asset5.count() # 9165374
#asset5.coalesce(1).write.option("inferSchema","true").csv("/public/ygong15/xerox/asset_natureV.1107",header = 'true')

### impute with maxppm

In [ ]:
df = asset5
df = df.withColumn('model_manf',F.concat_ws('<>',df.dim_model_nm,df.dim_model_manf_nm))
ppm_impute = F.udf(lambda x : model_manf_dict[x] if x in model_manf_dict.keys() else 0, types.IntegerType())
df2 = df.withColumn('max_ppm', F.when(df.model_manf.isin(model_manf_dict.keys()),ppm_impute(df.model_manf))\
              .otherwise(df.dim_model_ppm))
df2 = df2.drop('model_manf')
## Save to final version of maxppm1108
df2.coalesce(1).write.option("inferSchema","true").csv("/public/ygong15/xerox/maxppm1108",header = 'true')

### for Tableau Asset data
##### Asset Over Month _Tableau Asset Analysis Data (for tableau analysis)

In [ ]:
## left join asset_month and asset_nature into one dataframe 
asset_nature = df2
cond = [asset_month.dim_asset_sk == asset_nature.asset_id, asset_month.dim_acct_sk == asset_nature.account_id]
left_joinDF = asset_month.join(asset_nature, 
                               cond,
                               how='left')
left_joinDF = left_joinDF.drop('asset_id','account_id')
# left_joinDF.count() # 88430221

## Tableau Data for asset over month 
new_assetmonth = left_joinDF
scope = new_assetmonth.groupBy('month','dim_asset_inscope','dim_model_manf_nm','region','max_ppm')\
.agg(F.countDistinct('dim_asset_sk').alias('count'))
# scope.count() #104228
s =scope.toPandas()
s.to_csv('/scratch/ygong15/xerox/asset_overall_1118.csv', header=True, index=False)

## Tableau Data for Client over Month 
new_assetmonth = left_joinDF
client = new_assetmonth.filter(new_assetmonth.max_ppm > 0).groupBy('month')\
.agg(F.countDistinct('dim_acct_sk').alias('count')).orderBy('month')
c =client.toPandas()

a = new_assetmonth.filter(new_assetmonth.max_ppm > 0).groupBy('month')\
.agg(F.countDistinct('dim_asset_sk').alias('asset_cnt')).orderBy('month')
a_pd =a.toPandas()
c_df = pd.merge(c, a_pd, how='left', on='month')
c_df.to_csv('/scratch/ygong15/xerox/account_overtime.csv', header=True, index=False)

## Final step for getting final processed dataset 
### Final processed data will generate in part 2


###### 1. volume_with_ppm_11_11.csv
###### 2. inscope_11_11.csv and outscope_11_11.csv

###### 3. asset_for_type.csv 
###### 4. black_outscope.csv / color_outscope.csv / black_inscope.csv / color_inscope.csv (csv for Pandas)

### load data got above

In [ ]:
volume_has_out_scope = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
inferschema='true').load('/public/xye6/Xerox/volume_has_out_scope/volume_has_out_scope.csv')

In [ ]:
impute_data = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
inferschema='true').load('/public/ygong15/xerox/maxppm1108.csv')

### Join two table, Volume data get the ppm column (save as volume_with_ppm_11_11.csv)

In [ ]:
volume_with_maxppm=volume_has_out_scope.join(impute_data,['dim_asset_sk','dim_acct_sk'], how='left')

volume_with_maxppm=volume_with_maxppm.withColumn("max_ppm", volume_with_maxppm["max_ppm"].cast("long"))
volume_with_maxppm=volume_with_maxppm.withColumn("dim_model_ppm", volume_with_maxppm["dim_model_ppm"].cast("long"))

volume_with_maxppm=volume_with_maxppm.withColumn("month_volume_by_maxppm",volume_with_maxppm["max_ppm"]*60*24*30)
volume_with_maxppm=volume_with_maxppm.withColumn("month_volume_by_ppm",volume_with_maxppm["dim_model_ppm"]*60*24*30)

volume_with_maxppm=volume_with_maxppm.withColumn("uti_maxppm",volume_with_maxppm["Volume"]/volume_with_maxppm["month_volume_by_maxppm"])
volume_with_maxppm=volume_with_maxppm.withColumn("uti_normalppm",volume_with_maxppm["Volume"]/volume_with_maxppm["month_volume_by_ppm"])

tmp=volume_with_maxppm.withColumnRenamed('dim_meter_types_sk','meter_type')
tmp=tmp.withColumnRenamed('dim_asset_inscope','inscope')
volume_with_maxppm=tmp.withColumnRenamed('dim_asset_isdeleted','isdeleted')


targetDf = volume_with_maxppm.withColumn("Volume", when((volume_with_maxppm.uti_maxppm >1.0) | 
                                                        (volume_with_maxppm.uti_maxppm < 0.0) | (volume_with_maxppm.uti_maxppm.isNull()), 0).otherwise(volume_with_maxppm.Volume))


In [ ]:
#targetDf.coalesce(1).write.option("inferSchema","true").csv("/public/xye6/Xerox/volume_with_ppm_11_11",header = 'true')

### Get inscope volume table and outscope volume table ( inscope_11_11.csv and outscope_11_11.csv)

In [ ]:
# volume_11_11 = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
# inferschema='true').load('/public/xye6/Xerox/volume_with_ppm_11_11/volume_11_11.csv')

In [ ]:
volume_11_11=targetDf

In [ ]:
join_with_type=volume_11_11.select('dim_asset_sk','dim_acct_sk','meter_type','dim_invoice_dt_sk','Volume','inscope','isdeleted')

join_with_type=join_with_type.withColumn("dim_invoice_dt_sk", join_with_type["dim_invoice_dt_sk"].cast(StringType()))

join_with_type = join_with_type.withColumn('dim_invoice_dt_sk', 
                   to_date(unix_timestamp(col('dim_invoice_dt_sk'), 'yyyyMMdd').cast("timestamp")))

tmp=join_with_type.withColumn("month",month("dim_invoice_dt_sk"))
join_with_type=tmp.withColumn("day",dayofmonth("dim_invoice_dt_sk"))

join_with_type=join_with_type.withColumnRenamed('meter_type','dim_meter_types_sk')
join_with_type=join_with_type.withColumnRenamed('inscope','dim_asset_inscope')
join_with_type=join_with_type.withColumnRenamed('isdeleted','dim_asset_isdeleted')

New_vol = join_with_type.select([
    col('dim_asset_sk'),
    col('dim_acct_sk'),
    col('dim_asset_inscope'),
    col('Volume'),
    col('month'),
    col('dim_meter_types_sk').alias('meter')
])

vol_inscope=New_vol.filter(New_vol.dim_asset_inscope==1)
vol_outscope=New_vol.filter(New_vol.dim_asset_inscope==0)


In [ ]:
#vol_inscope.coalesce(1).write.option("inferSchema","true").csv("/public/xye6/Xerox/inscope_11_11.csv",header="true")
#vol_outscope.coalesce(1).write.option("inferSchema","true").csv("/public/xye6/Xerox/outscope_11_11.csv",header="true")

### Get asset_for_type.csv

In [ ]:
New_ass = asset.select([
    col('dim_asset_sk').alias('asset_sk'), 
    col('dim_acct_sk').alias('acct_sk'), 
    col('dim_model_nm').alias('model_nm'),
   # col('dim_asset_inscope'),
    col('dim_modified_dt_sk')])

New_ass = New_ass.withColumn('dim_modified_dt_sk', to_date(unix_timestamp(col('dim_modified_dt_sk'), 'yyyyMMdd').cast("timestamp")))
New_ass=New_ass.withColumn("Ass_month",month("dim_modified_dt_sk")).drop('dim_modified_dt_sk')

In [ ]:
#New_ass.coalesce(1).write.option("inferSchema","true").csv("/public/xye6/Xerox/asset_for_type.csv",header="true")

### Type rule function

In [ ]:
def _join(New_ass, New_vol):
    
    final_table = New_vol.join(New_ass, \
                               (New_ass.asset_sk == New_vol.dim_asset_sk) & (New_ass.acct_sk == New_vol.dim_acct_sk) & (New_ass.Ass_month == New_vol.month), 'left').drop('asset_sk', 'acct_sk', 'Ass_month')

    # The ones with ColorQube
    
    cq = final_table.where(col('model_nm').like('%ColorQube%'))
    # The ones without ColorQube but with color
    co = final_table.where((col('model_nm').like('%Color%')) & (~ (col('model_nm').like('%ColorQube%'))))
    # All the others
    bo = final_table.where((~(col('model_nm').like('%Color%'))) & (~ (col('model_nm').like('%ColorQube%'))))
    
    return cq, co, bo

def monthly(mon, cq, co, bo):
# ColorQube
    cq_mon = cq.filter(cq['month'] == mon)
    ### Black and white
    cq_mon_bw = cq_mon.filter((cq_mon['meter'] == 35) | (cq_mon['meter'] == 32))
    cq_mon_bw = cq_mon_bw.groupby('dim_asset_sk', 'dim_acct_sk', 'month').agg(F.sum('Volume').alias('BlackVolume'))
    ### Color
    cq_mon_col = cq_mon.filter((cq_mon['meter'] == 30) | (cq_mon['meter'] == 33) | (cq_mon['meter'] == 31) | (cq_mon['meter'] == 34))
    cq_mon_col = cq_mon_col.groupby('dim_asset_sk', 'dim_acct_sk', 'month').agg(F.sum('Volume').alias('ColorVolume'))

    # Color
    ## Color
    co_mon = co.filter(co['month'] == mon)
    ### Black and white
    co_mon_bw = co_mon.filter(co_mon['meter'] == 2)
    co_mon_bw = co_mon_bw.groupby('dim_asset_sk', 'dim_acct_sk', 'month').agg(F.sum('Volume').alias('BlackVolume'))
    ### Color
    co_mon_col = co_mon.filter(co_mon['meter'] == 3)
    co_mon_col = co_mon_col.groupby('dim_asset_sk', 'dim_acct_sk', 'month').agg(F.sum('Volume').alias('ColorVolume'))

    # All the others
    bo_mon = bo.filter(bo['month'] == mon)
    ### Black and white
    bo_mon_bw = bo_mon.filter(bo_mon['meter'] == 1)
    bo_mon_bw = bo_mon_bw.groupby('dim_asset_sk', 'dim_acct_sk', 'month').agg(F.sum('Volume').alias('BlackVolume'))

    # Concatenate
    total_bw = cq_mon_bw.union(co_mon_bw).union(bo_mon_bw)
    total_col = cq_mon_col.union(co_mon_col)
    return total_bw, total_col

def _get(New_ass, New_vol):
    cq, co, bo = _join(New_ass, New_vol)
    final_bw, final_col = monthly(1, cq, co, bo)
    
    for mon in range(2,13):
        bw, col = monthly(mon, cq, co, bo)
        final_bw=final_bw.union(bw)
        final_col=final_col.union(col)
        
    return final_bw, final_col

### Process data in Type rule part (inscope_11_11.csv | outscope_11_11.csv | asset_for_type.csv)

In [ ]:
# vol_inscope = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
# inferschema='true').load('/public/xye6/Xerox/inscope_11_11/inscope_11_11.csv')

# vol_outscope = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
# inferschema='true').load('/public/xye6/Xerox/outscope_11_11/outscope_11_11.csv')

# asset_for_type = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
# inferschema='true').load('/public/xye6/Xerox/asset_for_type.csv')


asset_for_type=New_ass


In [ ]:
black_in,color_in=_get(asset_for_type,vol_inscope)
black_out,color_out=_get(asset_for_type,vol_outscope)

### Get black_outscope.csv / color_outscope.csv / black_inscope.csv / color_inscope.csv  (csv for Pandas)

In [ ]:
black_outscope=black_out.toPandas()
black_outscope.to_csv('/public/xye6/Xerox/black_outscope.csv', sep='\t')

color_outscope=color_out.toPandas()
color_outscope.to_csv('/public/xye6/Xerox/color_outscope.csv', sep='\t')

black_inscope=black_in.toPandas()
black_inscope.to_csv('/public/xye6/Xerox/black_inscope.csv', sep='\t')

color_inscope=color_in.toPandas()
color_inscope.to_csv('/public/xye6/Xerox/color_inscope.csv', sep='\t')

## Generate volume_location.csv 
### for further analysis

In [ ]:
## left join asset_month and asset_nature into one dataframe 
asset_nature = df2
cond = [asset_month.dim_asset_sk == asset_nature.asset_id, asset_month.dim_acct_sk == asset_nature.account_id]
left_joinDF = asset_month.join(asset_nature, 
                               cond,
                               how='left')
left_joinDF = left_joinDF.drop('asset_id','account_id')
# left_joinDF.count() # 88430221

## create Tableau Data 
new_assetmonth = left_joinDF
scope = new_assetmonth.groupBy('month','dim_asset_inscope','dim_model_manf_nm','region','max_ppm')\
.agg(F.countDistinct('dim_asset_sk').alias('count'))
# scope.count() #104228
s =scope.toPandas()
# s.to_csv('/scratch/ygong15/xerox/asset_overall_1118.csv', header=True, index=False)

In [ ]:
volume_with_maxppm = sqlContext.read.format('com.databricks.spark.csv').options(header='true', 
inferschema='true').load('/public/xye6/Xerox/volume_with_ppm_1107/volume_with_ppm_1107.csv')

volume_with_maxppm.filter(volume_with_maxppm.dim_model_nm.isNull()).show()

In [ ]:
volume_locinfo=volume_with_maxppm.select('dim_asset_sk','dim_acct_sk','dim_model_nm','dim_model_manf_nm','dim_model_manf_is_xerox','dim_loc_city','country_iso3',
                          'region','state','max_ppm','dim_model_ppm')

In [ ]:
volume_locinfo=volume_locinfo.dropDuplicates(['dim_asset_sk','dim_acct_sk','dim_model_nm','dim_model_manf_nm'])

volume_location=volume_locinfo.toPandas()

In [ ]:
volume_location.to_csv('/public/xye6/Xerox/volume_location.csv', sep='\t',encoding='utf8')